In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
PATH_TO_DATA = '../data/'
NUMBER_OF_PREVIOUS_MONTHS = 3

In [20]:
sales_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'sales_train.csv'))

In [21]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [22]:
sales_train = (
    sales_train.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price': np.mean, 'item_cnt_day': np.sum}).reset_index()
)

In [23]:
sales_train.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,0,32,221.0,6.0
1,0,0,33,347.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,128.5,2.0


In [24]:
sales_train['date_block_num'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])

In [37]:
32 - 3

29

In [38]:
29 * 0.7

20.299999999999997

### Train - val - test  
* train is the first 20 months
* val is the second 9 months
* test is the last month

In [58]:
def make_dataset(fisrt_month: int, last_month: int) -> pd.DataFrame:
    # for valid_month in range(3, 33):
    total_dataset = None
    for valid_month in range(fisrt_month, last_month):
        valid_sales = sales_train[sales_train['date_block_num'] == valid_month].copy(deep=True)
        valid_sales = valid_sales[['shop_id', 'item_id', 'item_cnt_day']]
        total_month_dataset = valid_sales
        for j in range(1, NUMBER_OF_PREVIOUS_MONTHS+1):
            train_month = valid_month - j
            train_sales = sales_train[sales_train['date_block_num'] == train_month].copy(deep=True)
            train_sales = train_sales[['shop_id', 'item_id', 'item_price', 'item_cnt_day']]
            train_sales.rename(columns={
                'item_price': f'item_price_lag_{j}',
                'item_cnt_day': f'item_cnt_day_lag_{j}'
            }, inplace=True)
            
            total_month_dataset = pd.merge(total_month_dataset, train_sales, on=['shop_id', 'item_id'], how='left')
        
        cols = list(total_month_dataset.columns.values)
        cols.pop(cols.index('item_cnt_day'))
        cols.append('item_cnt_day')
        total_month_dataset = total_month_dataset[cols]

        if total_dataset is None:
            total_dataset = total_month_dataset
        else:
            total_dataset = pd.concat([total_dataset, total_month_dataset], axis=1)

    return total_dataset

In [59]:
%%time
make_dataset(3, 33)

   shop_id  item_id  item_price_lag_1  item_cnt_day_lag_1  item_price_lag_2  \
0      2.0     31.0             698.5                 1.0             699.0   
1      2.0    464.0               NaN                 NaN               NaN   
2      2.0    482.0            3300.0                 1.0            3300.0   
3      2.0    484.0             300.0                 2.0             300.0   
4      2.0    485.0               NaN                 NaN               NaN   

   item_cnt_day_lag_2  item_price_lag_3  item_cnt_day_lag_3  item_cnt_day  \
0                 4.0               NaN                 NaN           1.0   
1                 NaN               NaN                 NaN           1.0   
2                 1.0            3300.0                 1.0           1.0   
3                 4.0             300.0                 2.0           3.0   
4                 NaN               NaN                 NaN           2.0   

   shop_id      ...       item_cnt_day  shop_id  item_id  item